<a href="https://colab.research.google.com/github/Prit-P2/AI_agent_of_coding/blob/main/Presentation_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Colab secrets. Details: {e}")

✅ Gemini API key setup complete.


### 1.3: Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
!pip install python-pptx

In [ ]:
from pptx import Presentation

def create_slides_from_content(title="Presentation Title", content="", lines_per_slide=10):
    """
    Creates a PPTX and automatically overflows content to new slides.
    """
    prs = Presentation()

    # Split content into a list of lines/paragraphs
    lines = content.split('\n')

    # Group lines into chunks based on lines_per_slide
    # This creates a list of lists: [[line1, line2], [line3, line4]]
    chunks = [lines[i:i + lines_per_slide] for i in range(0, len(lines), lines_per_slide)]

    for index, chunk in enumerate(chunks):
        # Use Title and Content layout (Index 1)
        slide_layout = prs.slide_layouts[1]
        slide = prs.slides.add_slide(slide_layout)

        # Set the Title (Add "cont." for subsequent slides)
        title_shape = slide.shapes.title
        title_shape.text = f"{title} (Part {index + 1})" if index > 0 else title

        # Add the chunk of text to the body
        body_shape = slide.placeholders[1]
        tf = body_shape.text_frame
        tf.word_wrap = True # Ensure text wraps within the box

        # Join the chunk lines back into a single string for the text frame
        tf.text = "\n".join(chunk)

    save_path = "multi_slide_presentation.pptx"
    prs.save(save_path)
    print(f"Success! Created {len(chunks)} slides.")
    return {"slides_created": len(chunks)}

# Example Usage:
long_text = "\n".join([f"This is important point number {i}" for i in range(1, 26)])
create_slides_from_content("My Automated Report", long_text, lines_per_slide=8)

Success! Created 4 slides.


{'slides_created': 4}

In [ ]:
create_slides_from_content()

Success! Created 1 slides.


{'slides_created': 1}

In [1]:
# Currency agent with custom function tools
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""You are a smart ppt generator agent your task:

    1.Use 'create_new_slide(argument1,argument2)' first argument title argument2 content
    2.You will gine title on the basic of topic provide in quary
    3.You will use Gemini to generate a content and give that content in second argunlment
    4.If it genterate then give sussfully created
    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[create_slides_from_content],
)

print("✅ Currency agent created with custom function tools")

NameError: name 'LlmAgent' is not defined

In [ ]:
# Test the currency agent
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug(
    "Presentation topic is AI Agent"
)


 ### Created new session: debug_session_id

User > Presentation topic is AI Agent


Success! Created 3 slides.
currency_agent > Successfully created 3 slides.


In [ ]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.

     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.

     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.

    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

In [ ]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

In [ ]:
# Define a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

In [ ]:
# Test the enhanced agent
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)